In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mplcyberpunk
plt.style.use('cyberpunk')


In [24]:
data_raw_train=pd.read_csv(r'C:\Users\Ashinth\Documents\courses\Kaggle\competetions\spaceship-titanic\train.csv')
data_raw_test=pd.read_csv(r'C:\Users\Ashinth\Documents\courses\Kaggle\competetions\spaceship-titanic\test.csv')

In [25]:
data_raw_train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


# Preprocessing

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

In [27]:
data_raw_train[['Cabin1','Cabin2','Cabin3']]=data_raw_train['Cabin'].str.split('/',expand=True)
data_raw_train.drop(['Cabin'],axis=1,inplace=True)
data_raw_test[['Cabin1','Cabin2','Cabin3']]=data_raw_test['Cabin'].str.split('/',expand=True)
data_raw_test.drop(['Cabin'],axis=1,inplace=True)
data_raw_train.Cabin2.astype('float64')
data_raw_test.Cabin2.astype('float64')

0          3.0
1          4.0
2          0.0
3          1.0
4          5.0
         ...  
4272    1496.0
4273       NaN
4274     296.0
4275     297.0
4276    1498.0
Name: Cabin2, Length: 4277, dtype: float64

In [28]:
cat_cols=[cols for cols in data_raw_train.columns if data_raw_train[cols].dtype in ['object','bool'] and data_raw_train[cols].nunique()<=10]
num_cols=[cols for cols in data_raw_train.columns if data_raw_train[cols].dtype in ['int64','float64']]
unique_vals=dict()

for val in num_cols:
    unique_vals[val]=data_raw_train[val].nunique()
unique_vals

ans_col=data_raw_test.PassengerId
#cols=cat_cols+num_cols
#data_raw_train=data_raw_train[cols]
data_raw_train.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin1,Cabin2,Cabin3
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


__Eliminating the rows with null values__  
After which imputing and encoding are done

In [29]:
#data_raw_train['Transported'].isnull().unique()
# From this we see that there are no null values in the target column, so no need to drop any rows 
y=data_raw_train['Transported']
data_raw_train.drop('Transported',axis=1,inplace=True)
cat_cols.remove('Transported')
cols=cat_cols+num_cols
X=data_raw_train
X_test=data_raw_test

In [30]:
X_train,X_val, y_train,y_val=train_test_split(X,y,test_size=0.2,train_size=0.8,random_state=0)
id_train=pd.Series( X_train.PassengerId,name='PassengerId')
id_val= pd.Series(X_val.PassengerId,name='PassengerId')

X_train=X_train[cols]
X_val=X_val[cols]

In [31]:
categorical_part= Pipeline(steps=[('imputing',SimpleImputer(strategy='most_frequent')),('onehot',OneHotEncoder(sparse=False,handle_unknown='ignore'))])

numerical_part=SimpleImputer(strategy='mean')

preprocessing=ColumnTransformer(transformers=[('categorical',categorical_part,cat_cols),('numerical',numerical_part,num_cols)])
#preprocessing.fit(X_train)

In [32]:
# scaling the numerical data
num_scale=MinMaxScaler()
num_scale.fit(X_train[num_cols])
X_train_scaled=X_train.copy()
X_val_scaled=X_val.copy()
X_train_scaled[num_cols]=num_scale.transform(X_train[num_cols])
X_val_scaled[num_cols]=num_scale.transform(X_val[num_cols])

In [33]:
from sklearn.preprocessing import LabelEncoder

indices_train=list(y_train.index)
indices_val=list(y_val.index)
enc=LabelEncoder()
y_train_encoded=enc.fit_transform(y_train)
y_val_encoded=enc.transform(y_val)

In [34]:
X_train_scaled.head()

,HomePlanet,CryoSleep,Destination,VIP,Cabin1,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
4278,Europa,False,55 Cancri e,False,C,S,0.683544,0.000000,0.020164,0.000000,0.820482,0.115982
5971,Earth,False,TRAPPIST-1e,False,F,P,0.253165,0.000000,0.000721,0.000043,0.037476,0.000000
464,Mars,False,TRAPPIST-1e,False,F,S,0.544304,0.127103,0.000000,0.002001,0.001561,0.000000
4475,Earth,False,TRAPPIST-1e,False,F,S,0.303797,0.012913,0.000000,0.020262,0.097459,0.002196
8469,Europa,True,55 Cancri e,False,C,S,0.316456,0.000000,0.000000,0.000000,0.000000,0.000000


## Creating model and evaluating the pipeline  
__And then assesing the predictors that are important__

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
#def forest(val):
model=SVC(C=200,random_state=0)
#    return model

In [36]:
X_train_scaled.dtypes

HomePlanet       object
CryoSleep        object
Destination      object
VIP              object
Cabin1           object
Cabin3           object
Age             float64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object

In [37]:
from sklearn.metrics import accuracy_score
acc=list()
#n_estimators=[5,7,10,15,20,25]
#for i in n_estimators:
my_pipe= Pipeline(steps=[('preprocessor',preprocessing),('model',model)])
my_pipe.fit(X_train_scaled,y_train_encoded)
predictions=my_pipe.predict(X_val_scaled)
acc.append(accuracy_score(predictions,y_val_encoded))
print(acc)


[0.7958596894767107]


In [38]:
K=pd.Series(predictions,name='Transported')

In [39]:
result= pd.merge(id_val,K,right_index=True,left_index=True)


In [40]:
vals=dict(zip(n_estimators,acc))
plt.figure(figsize=[8,8])
plt.plot(n_estimators,acc)
plt.title("Accuracy of the validation data with respect to the number of trees")
plt.xlabel("number of trees(n_estimators)")
plt.ylabel("model accuracy")
print(vals)

NameError: name 'n_estimators' is not defined

Checking the results for the test data

In [41]:
X_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin1,Cabin2,Cabin3
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S


In [42]:
id_test=pd.Series(X_test.PassengerId,name='PassengerId')
X_test=X_test[cols]
X_test_scaled=X_test.copy()

X_test_scaled[num_cols]=num_scale.transform(X_test[num_cols])
X_test_scaled.head()

,HomePlanet,CryoSleep,Destination,VIP,Cabin1,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,TRAPPIST-1e,False,G,S,0.341772,0.000000,0.000000,0.00000,0.000000,0.000000
1,Earth,False,TRAPPIST-1e,False,F,S,0.240506,0.000000,0.000325,0.00000,0.152003,0.000000
2,Europa,True,55 Cancri e,False,C,S,0.392405,0.000000,0.000000,0.00000,0.000000,0.000000
3,Europa,False,TRAPPIST-1e,False,C,S,0.481013,0.000000,0.239945,0.00000,0.009746,0.024241
4,Earth,False,TRAPPIST-1e,False,F,S,0.253165,0.000698,0.000000,0.02703,0.000000,0.000000


In [43]:
X_test_scaled.head()


,HomePlanet,CryoSleep,Destination,VIP,Cabin1,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,TRAPPIST-1e,False,G,S,0.341772,0.000000,0.000000,0.00000,0.000000,0.000000
1,Earth,False,TRAPPIST-1e,False,F,S,0.240506,0.000000,0.000325,0.00000,0.152003,0.000000
2,Europa,True,55 Cancri e,False,C,S,0.392405,0.000000,0.000000,0.00000,0.000000,0.000000
3,Europa,False,TRAPPIST-1e,False,C,S,0.481013,0.000000,0.239945,0.00000,0.009746,0.024241
4,Earth,False,TRAPPIST-1e,False,F,S,0.253165,0.000698,0.000000,0.02703,0.000000,0.000000


In [44]:
answer_predictions=pd.Series(my_pipe.predict(X_test_scaled),name='Transported')

__Have to convert the 1's and zeros to true/false.__

In [45]:
answer=list()
for i in range(len(answer_predictions)):
    if (answer_predictions.iloc[i]==1):
        answer.append(True)
    elif(answer_predictions.iloc[i]==0):
        answer.append(False)
answer=pd.Series(answer,name='Transported')
test_result=pd.merge(id_test,answer,right_index=True,left_index=True)
test_result.set_index('PassengerId')
test_result.head()
        

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [47]:
test_result.to_csv('Submission.csv',index=False)